In [1]:
#some notes
#1. Doesn't look like a player receive a curse is treated as an action 
#2. Should I add round end markers?

In [112]:
import pandas as pd
import pdb
import re
import nltk
from itertools import product
from nltk import ngrams
from nltk.probability import FreqDist
nltk.download('punkt')
from collections import Counter

#define number of NGrams to use
N = 2

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anthonyowen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data  = pd.read_csv("data/ActionsReduced_BMWG_vs_DW_GPM100.csv")
data = data[['GameID', 'Player', 'Round','Turn','ActionDescription']]
print(data)
print(data.columns)

       GameID  Player  Round  Turn              ActionDescription
0           2       0      0     0              End Current Phase
1           2       0      0     0              End Current Phase
2           2       1      0     1              End Current Phase
3           2       1      0     1    BuyCard: SILVER by player 1
4           2       0      1     0              End Current Phase
...       ...     ...    ...   ...                            ...
17005     201       0     18     0    BuyCard: ESTATE by player 0
17006     201       1     18     1               WITCH : Player 1
17007     201       1     18     1  BuyCard: PROVINCE by player 1
17008     201       0     19     0              End Current Phase
17009     201       0     19     0  BuyCard: PROVINCE by player 0

[17010 rows x 5 columns]
Index(['GameID', 'Player', 'Round', 'Turn', 'ActionDescription'], dtype='object')


In [92]:
#kingdom card types
card_types_SD = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY', 'THRONE_ROOM', 'WITCH',
                 'WORKSHOP', 'CURSE', 'PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types_SD_no_curse = ['ARTISAN', 'BANDIT', 'BUREAUCRAT', 'CHAPEL', 'FESTIVAL', 'GARDENS', 'SENTRY', 'THRONE_ROOM', 'WITCH',
                 'WORKSHOP','PROVINCE', 'DUCHY', 'ESTATE', 'GOLD', 'SILVER', 'COPPER']
card_types = card_types_SD_no_curse

In [93]:
#first format action decription
def format_action(action, cardtypes):
    #there are ... types of actions we need to format
    # 1. 'End Current Phase' goes to ECP
    # 2. 'BuyCard: CARDX by player Y' goes to BUYCARDX_Y
    # 3. 'CARDX : Player Y' goes to PLAYCARDX_Y
    
    #to identify these we use regular expressions
    pattern1 = re.compile(r'End Current Phase')
    pattern2 = re.compile(r'BuyCard: (' + '|'.join(cardtypes) + r') by player (0|1)')
    pattern3 = re.compile(r'(' + '|'.join(cardtypes) + r') : Player (0|1)')
   
    match1 = pattern1.match(action)
    match2 = None
    match3 = None
    if match1 == None:
        match2 = pattern2.match(action)
    if match2 == None:
        match3 =  pattern3.match(action)
        
    if match1 != None:
        formatted_action = 'ECF'
    elif match2 != None:
        matched_card = match2.group(1)
        player = match2.group(2)
        formatted_action = 'BUY' + matched_card #+ '_' + str(player)
    elif match3 != None:
        matched_card = match3.group(1)
        player = match3.group(2)
        formatted_action = 'PLAY' + matched_card #+ '_' + str(player)
    else:
        raise Exception("Can't match action description")
        
    return formatted_action

#print(format_action('GOLD : Player 1', card_types_SD))
#print(format_action('BuyCard: PROVINCE by player 1', card_types))
#print(format_action('End Current Phase', card_types_SD))

In [94]:
#process input file
data['ProcAction'] = data.apply(lambda row: format_action(row['ActionDescription'], card_types), axis = 1)
print(data)

       GameID  Player  Round  Turn              ActionDescription   ProcAction
0           2       0      0     0              End Current Phase          ECF
1           2       0      0     0              End Current Phase          ECF
2           2       1      0     1              End Current Phase          ECF
3           2       1      0     1    BuyCard: SILVER by player 1    BUYSILVER
4           2       0      1     0              End Current Phase          ECF
...       ...     ...    ...   ...                            ...          ...
17005     201       0     18     0    BuyCard: ESTATE by player 0    BUYESTATE
17006     201       1     18     1               WITCH : Player 1    PLAYWITCH
17007     201       1     18     1  BuyCard: PROVINCE by player 1  BUYPROVINCE
17008     201       0     19     0              End Current Phase          ECF
17009     201       0     19     0  BuyCard: PROVINCE by player 0  BUYPROVINCE

[17010 rows x 6 columns]


In [95]:
#extract a single player list of actions for a single game
trace_data = data[(data['GameID'] == 2) & ((data['Player'] == 0))]['ProcAction']

#next we need to convert into a single string to apply Ngram functions
trace_str = trace_data.str.cat(sep=' ')
print(trace_str)

ECF ECF ECF BUYWITCH ECF BUYSILVER PLAYWITCH BUYWITCH ECF BUYSILVER ECF BUYSILVER ECF BUYSILVER PLAYWITCH BUYSILVER ECF BUYSILVER ECF BUYGOLD PLAYWITCH BUYGOLD PLAYWITCH BUYPROVINCE PLAYWITCH BUYPROVINCE ECF BUYPROVINCE ECF BUYSILVER ECF BUYPROVINCE PLAYWITCH BUYPROVINCE ECF BUYSILVER ECF BUYSILVER PLAYWITCH BUYDUCHY


In [114]:
#next we need to extract N-grams
tokens = nltk.word_tokenize(trace_str)
trace_n_grams = list(ngrams(tokens, N))
print(trace_n_grams)

[('ECF', 'ECF'), ('ECF', 'ECF'), ('ECF', 'BUYWITCH'), ('BUYWITCH', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'PLAYWITCH'), ('PLAYWITCH', 'BUYWITCH'), ('BUYWITCH', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'PLAYWITCH'), ('PLAYWITCH', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYGOLD'), ('BUYGOLD', 'PLAYWITCH'), ('PLAYWITCH', 'BUYGOLD'), ('BUYGOLD', 'PLAYWITCH'), ('PLAYWITCH', 'BUYPROVINCE'), ('BUYPROVINCE', 'PLAYWITCH'), ('PLAYWITCH', 'BUYPROVINCE'), ('BUYPROVINCE', 'ECF'), ('ECF', 'BUYPROVINCE'), ('BUYPROVINCE', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYPROVINCE'), ('BUYPROVINCE', 'PLAYWITCH'), ('PLAYWITCH', 'BUYPROVINCE'), ('BUYPROVINCE', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'ECF'), ('ECF', 'BUYSILVER'), ('BUYSILVER', 'PLAYWITCH'), ('PLAYWITCH', 'BUYDUCHY')]


In [115]:
#calculate frequency distribution
freq_dist = FreqDist(trace_n_grams)
for key, value in freq_dist.items():
    print(f'{key}: {value}')
print("No of distinct ngrams observed: " + str(len(freq_dist.keys())))

('ECF', 'ECF'): 2
('ECF', 'BUYWITCH'): 1
('BUYWITCH', 'ECF'): 2
('ECF', 'BUYSILVER'): 8
('BUYSILVER', 'PLAYWITCH'): 3
('PLAYWITCH', 'BUYWITCH'): 1
('BUYSILVER', 'ECF'): 6
('PLAYWITCH', 'BUYSILVER'): 1
('ECF', 'BUYGOLD'): 1
('BUYGOLD', 'PLAYWITCH'): 2
('PLAYWITCH', 'BUYGOLD'): 1
('PLAYWITCH', 'BUYPROVINCE'): 3
('BUYPROVINCE', 'PLAYWITCH'): 2
('BUYPROVINCE', 'ECF'): 3
('ECF', 'BUYPROVINCE'): 2
('PLAYWITCH', 'BUYDUCHY'): 1
No of distinct ngrams observed: 16


In [109]:
#create list of all possible actions - note we are ignoring responses to opponents cards (for SD this means just 
#ignoring receiving curse cards

buy_actions = ['BUY' + str(card) for card in card_types]
play_actions = ['PLAY' + str(card) for card in card_types]
all_actions_list = ['ECF'] + buy_actions + play_actions
print("Action list length: " + str(all_actions_list))
print(all_actions_list)

#create list of all possible N-grams
all_ngrams_list = list(product(all_actions_list, repeat=N))
print(all_ngrams_list)
print("N-gram list length: " + str(len(all_ngrams_list)))

Action list length: ['ECF', 'BUYARTISAN', 'BUYBANDIT', 'BUYBUREAUCRAT', 'BUYCHAPEL', 'BUYFESTIVAL', 'BUYGARDENS', 'BUYSENTRY', 'BUYTHRONE_ROOM', 'BUYWITCH', 'BUYWORKSHOP', 'BUYPROVINCE', 'BUYDUCHY', 'BUYESTATE', 'BUYGOLD', 'BUYSILVER', 'BUYCOPPER', 'PLAYARTISAN', 'PLAYBANDIT', 'PLAYBUREAUCRAT', 'PLAYCHAPEL', 'PLAYFESTIVAL', 'PLAYGARDENS', 'PLAYSENTRY', 'PLAYTHRONE_ROOM', 'PLAYWITCH', 'PLAYWORKSHOP', 'PLAYPROVINCE', 'PLAYDUCHY', 'PLAYESTATE', 'PLAYGOLD', 'PLAYSILVER', 'PLAYCOPPER']
['ECF', 'BUYARTISAN', 'BUYBANDIT', 'BUYBUREAUCRAT', 'BUYCHAPEL', 'BUYFESTIVAL', 'BUYGARDENS', 'BUYSENTRY', 'BUYTHRONE_ROOM', 'BUYWITCH', 'BUYWORKSHOP', 'BUYPROVINCE', 'BUYDUCHY', 'BUYESTATE', 'BUYGOLD', 'BUYSILVER', 'BUYCOPPER', 'PLAYARTISAN', 'PLAYBANDIT', 'PLAYBUREAUCRAT', 'PLAYCHAPEL', 'PLAYFESTIVAL', 'PLAYGARDENS', 'PLAYSENTRY', 'PLAYTHRONE_ROOM', 'PLAYWITCH', 'PLAYWORKSHOP', 'PLAYPROVINCE', 'PLAYDUCHY', 'PLAYESTATE', 'PLAYGOLD', 'PLAYSILVER', 'PLAYCOPPER']
[('ECF', 'ECF'), ('ECF', 'BUYARTISAN'), ('ECF', 

In [122]:
# Compute the frequency of strings in list1 appearing in list2
frequency_counter = Counter(trace_n_grams)

# Get the frequency of each string from list1
result = {gram: frequency_counter.get(gram, 0) for gram in set(all_ngrams_list)}

#normalise each entry with the length of all possible n-grams
result_normalised = {key: value / (1.0*len(all_ngrams_list)) for key, value in result.items()}

#check sum of probabilities is one
total_prob = 0 
for key, value in result_normalised.items():
    total_prob += value

print(total_prob)

0.035812672176308534


In [ ]:
# Calculate probabilities
def calc_probabilities(n_grams_list, all_n_grams_list):
    

In [ ]:
#check probabilities sum to one